In [139]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import pandas as pd
from torch.utils.data import Dataset
from skimage import io
import matplotlib.pyplot as plt
import torch.nn.functional as F
import optuna

In [56]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# Lab for Week 5: Fitting a Feed Forward Neural Network

In Milestone 1 you chose a project topic you are interested in and found some datasets you could use. You also already built a [data loader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

Today, we will fit a baseline feed forward neural network to one of the datasets you chose by following the pytorch tutorials.

## Step 1: Define a Data Loader
You are encouraged to use one of the data loaders you built for milestone 1. Otherwise use the one described [here](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

Tip: If your targets are labels, you might want to use the lambda transformation described [here](https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html) to turn your target into one-hot vectors.

In [120]:
import torch
from torchvision import transforms
from PIL import Image
import os


transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize to 299x299, this is a better size for the type of images I'm using
    transforms.ToTensor(),          # Converting images to tensor
])


from sklearn.preprocessing import LabelEncoder
# Loding the dataset
fitzpatrick = 'https://raw.githubusercontent.com/mattgroh/fitzpatrick17k/refs/heads/main/fitzpatrick17k.csv'
df = pd.read_csv(fitzpatrick)
image_folder = '/Users/laureladams/Documents/School/spring2025/introtoADL/introToAppliedDeepLearning/images'
#Definnig the class
class CustomDataset(Dataset):
    def __init__(self, df, image_folder):
        self.image_folder = image_folder
        # Filter out rows with missing images
        valid_rows = []
        for idx, row in df.iterrows():
            image_path = os.path.join(image_folder, f"{row['md5hash']}.jpg")
            if os.path.exists(image_path):
                valid_rows.append(idx)
            else:
                print(f"Warning: Image not found: {image_path}")
        self.data = df.loc[valid_rows].reset_index(drop=True)

        self.features = self.data[['md5hash']].values
        self.labels = self.data['label'].values

        # Label encoding that will convert fitzpatrick string labels (skin tone classification) into integers
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_folder, f"{self.features[idx][0]}.jpg")
    
        if os.path.exists(image_path):
            img = Image.open(image_path).convert("RGB")  
            img = transform(img)  # Apply transformations to the images (resize + ToTensor)
        else:
            raise FileNotFoundError(f"Image not found: {image_path}") 
            #to check if the images are there or missing, because I was having issues with this originally
    
        label = torch.tensor(self.labels[idx], dtype=torch.long)  # To convert label to tensor
    
        return img, label  # To make sure the image is a tensor before returning, was getting errors from this 
    


from sklearn.model_selection import train_test_split
# Making training and testing dataset!
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = CustomDataset(df_train, image_folder)
test_dataset = CustomDataset(df_test, image_folder)

print(len(train_dataset)) # just to make sure this worked
print(len(test_dataset))



13216
3302


In [84]:
# Used this cell to process the images, resize, and put them in a new folder

# # Define paths
# input_folder = "/Users/laureladams/Documents/School/spring2025/introtoADL/introToAppliedDeepLearning/images"
# output_folder = "/Users/laureladams/Documents/School/spring2025/introtoADL/introToAppliedDeepLearning/processed_images"

# # Ensure output folder exists
# os.makedirs(output_folder, exist_ok=True)

# # Define transformation
# transform = transforms.Compose([
#     transforms.Resize((299, 299))  # Resize to 299x299
# ])

# # Process images
# for filename in os.listdir(input_folder):
#     if filename.endswith(".jpg") or filename.endswith(".png"):  # Add other formats if needed
#         img_path = os.path.join(input_folder, filename)
#         img = Image.open(img_path)
#         img = transform(img)  # Apply resize transform
#         img.save(os.path.join(output_folder, filename))  # Save resized image

# print("Processing complete! All images are resized and saved in 'processed_images'.")



In [121]:
#  DataLoader that I will use for training and testing
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Example: Print the first batch
for batch_X, batch_y in train_loader:
    print(batch_X.shape, batch_y.shape)  
    break


torch.Size([8, 3, 299, 299]) torch.Size([8])


In [122]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from keras.models import Model
from keras.preprocessing import image
from scipy.spatial import distance_matrix
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3

In [123]:
import os

# Original images folder
working_path = '/Users/laureladams/Documents/School/spring2025/introtoADL/introToAppliedDeepLearning/images'

# New folder for processed images
save_path = '/Users/laureladams/Documents/School/spring2025/introtoADL/introToAppliedDeepLearning/processed_images'

# Create the new folder if it doesn't already exist
os.makedirs(save_path, exist_ok=True)

# Verify the folder was created
print(f"Processed images will be saved to: {save_path}")


Processed images will be saved to: /Users/laureladams/Documents/School/spring2025/introtoADL/introToAppliedDeepLearning/processed_images


In [124]:
print('length of training dataset:')
print(len(train_dataset))
print('length of test dataset:')
print(len(test_dataset))

length of training dataset:
13216
length of test dataset:
3302


In [125]:
train_dataloader = DataLoader(train_dataset)
test_dataloader = DataLoader(test_dataset)

In [126]:
for x, y in train_dataloader:
    print(x)
    break

tensor([[[[0.4510, 0.3725, 0.3569,  ..., 0.5176, 0.5059, 0.4902],
          [0.3333, 0.2941, 0.2824,  ..., 0.5333, 0.5176, 0.5137],
          [0.3059, 0.3137, 0.2627,  ..., 0.5412, 0.5451, 0.5255],
          ...,
          [0.0314, 0.0784, 0.1059,  ..., 0.0078, 0.0039, 0.0039],
          [0.0157, 0.0667, 0.1098,  ..., 0.0039, 0.0000, 0.0000],
          [0.0431, 0.1020, 0.1412,  ..., 0.0039, 0.0039, 0.0039]],

         [[0.2706, 0.2196, 0.2157,  ..., 0.2588, 0.2549, 0.2431],
          [0.1804, 0.1451, 0.1490,  ..., 0.2784, 0.2745, 0.2706],
          [0.1529, 0.1373, 0.1059,  ..., 0.2980, 0.3059, 0.2902],
          ...,
          [0.5294, 0.5255, 0.5294,  ..., 0.5922, 0.5843, 0.5843],
          [0.5255, 0.5216, 0.5255,  ..., 0.5922, 0.5961, 0.5922],
          [0.5176, 0.5216, 0.5294,  ..., 0.5882, 0.5922, 0.5882]],

         [[0.2157, 0.1843, 0.1843,  ..., 0.1961, 0.1882, 0.1765],
          [0.1451, 0.1176, 0.1216,  ..., 0.2157, 0.2039, 0.2000],
          [0.1176, 0.0980, 0.0667,  ..., 0

## Step 2: Define a Model
Follow [this tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) to define a model. Make sure that the input dimension fits the dimensionality of your data, and the output dimension fits the dimensionality of your targets.

In [127]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        # Define the fully connected layers
        self.fc1 = nn.Linear(299 * 299 * 3, 512)  # Updated the input size to match the flattened image size
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 114)  # There are 114 output classes (skin types/diseases represented)

    def forward(self, x):
       
        x = torch.flatten(x, 1)  # Flatten the image

        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Output layer (no activation function, logits)
        
        return x


In [128]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=268203, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=114, bias=True)
)


## Step 3: Train Model

Follow the tutorial [here](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) to train your model.

In [129]:
# Defining hyperparameters for optimization
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [130]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    
    """
    Runs one full training epoch on the given model using the provided dataloader.

    Parameters:
        dataloader (torch.utils.data.DataLoader): DataLoader providing batches of training data (inputs and labels).
        model (torch.nn.Module): The PyTorch model to be trained.
        loss_fn (function): The loss function used to compute the error between predictions and true labels.
        optimizer (torch.optim.Optimizer): The optimizer used to update the model parameters.

    Behavior:
        - Sets the model to training mode (important for layers like dropout and batch normalization).
        - Iterates over the data batches:
            - Computes the model's predictions.
            - Calculates the loss.
            - Performs backpropagation and updates model weights.
        - Prints progress every 100 batches, showing the current loss and the number of samples processed.

    Returns:
        None
    """
    
#    TODO: Implement this function

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print("Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    """
    Evaluates the model's performance on a test dataset.

    Parameters:
        dataloader (torch.utils.data.DataLoader): DataLoader providing batches of test data (inputs and labels).
        model (torch.nn.Module): The PyTorch model to be evaluated.
        loss_fn (function): The loss function used to compute the error between predictions and true labels.

    Behavior:
        - Sets the model to evaluation mode (which disables behaviors like dropout).
        - Iterates over the test data without computing gradients (using torch.no_grad() for efficiency).
        - Accumulates the total loss and counts the number of correct predictions.
        - Computes the average loss and overall accuracy.
        - Prints the test accuracy and average loss.

    Returns:
        None
    """ 
    # TODO: Implement this function
    


In [149]:
# Train the model for 3 different hyper parameter settings (e.g. different learning rates, different loss functions that make sense for your data, etc.)

# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# epochs = 10
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train_loop(train_dataloader, model, loss_fn, optimizer)
#     test_loop(test_dataloader, model, loss_fn)
# print("Done!")

# Assuming `train_loop` and `test_loop` are defined to train and evaluate your model

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 5  # reduced to 5 because 10 was taking forever

val_losses = []

for epoch in range(epochs):
    train_loop(train_loader, model, loss_fn, optimizer)
    val_loss = test_loop(test_loader, model, loss_fn)
    
    val_losses.append(val_loss)  # Wanted to store the validation loss so I'd be able to see it

print("Validation losses over epochs:", val_losses)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x262144 and 1048576x114)

In [ ]:
#opimization

# Define the model, loss function, and optimizer here (using your existing model)
def create_model():
    # Example: Define your model architecture (change this to your actual model)
    model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(64*128*128, 114)  # Change dimensions based on your input size
    )
    return model

def objective(trial):
    # Hyperparameter tuning
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 16, 128)
    momentum = trial.suggest_uniform('momentum', 0.5, 0.9)
    
    # Recreate model and dataloaders for each trial
    model = create_model()
    loss_fn = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

## Step 4: Record results

Results are typically recorded in a table. For inspiration, check out the famous [Attention is All You Need paper](https://arxiv.org/abs/1706.03762). This paper first introduced the transformer architecture we will learn about later this semseter and has been highly influential in deep learnin. Check out how results are reported in Tables 2, 3, and 4.
Note that because here the transformers are used for text generation results are reported using the BLEU score (the higher the better). 

You should create your own result tables to record how different hyperparameter settings affect performance. Make sure to record test accuracy, not traingin accuracy!